In [1]:
import findspark

In [2]:
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName("WordCount").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/22 08:15:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc=spark.sparkContext

In [5]:
rdd=sc.parallelize([1,2,3,4,5])

In [6]:
resultrdd=rdd.map(lambda x:x*2)
resultrdd.collect()

[2, 4, 6, 8, 10]

In [7]:
resultrdd1=rdd.flatMap(lambda x:(x,x*2))
resultrdd1.collect()

[1, 2, 2, 4, 3, 6, 4, 8, 5, 10]

In [8]:
resultrdd2=rdd.filter(lambda x:x % 2==0)
resultrdd2.collect()

[2, 4]

In [9]:
resultrdd3=rdd.map(lambda x:(x,x*2))
resultrdd3.collect()

[(1, 2), (2, 4), (3, 6), (4, 8), (5, 10)]

In [10]:
rdd2=sc.parallelize([(1,2),(3,4),(1,6),(2,3)])
res4=rdd2.reduceByKey(lambda x,y: x+y)
res4.collect()

[(2, 3), (1, 8), (3, 4)]

In [11]:
res5=rdd2.groupByKey()
res5.collect()

[(2, <pyspark.resultiterable.ResultIterable at 0x7f905416d410>),
 (1, <pyspark.resultiterable.ResultIterable at 0x7f905415d550>),
 (3, <pyspark.resultiterable.ResultIterable at 0x7f905416d510>)]

In [12]:
for key, values in res5.collect():
    print(f"Key: {key}, Values: {list(values)}")

Key: 2, Values: [3]
Key: 1, Values: [2, 6]
Key: 3, Values: [4]


In [13]:
words_list=["this","is","a","sample","text","document","for","word","count","example","word","count","is"]

In [14]:
rdd=sc.parallelize(words_list)

In [15]:
word_count = rdd.flatMap(lambda x: x.split(","))
word_count.collect()

['this',
 'is',
 'a',
 'sample',
 'text',
 'document',
 'for',
 'word',
 'count',
 'example',
 'word',
 'count',
 'is']

In [16]:
wc = rdd.map(lambda x:(x,1)).reduceByKey(lambda a,b: a+b)
wc.collect()

[('this', 1),
 ('sample', 1),
 ('text', 1),
 ('for', 1),
 ('word', 2),
 ('is', 2),
 ('a', 1),
 ('document', 1),
 ('count', 2),
 ('example', 1)]

In [17]:
purchaserdd=sc.textFile("/home/labuser/Desktop/Pandas_datasets/purchases.csv")
purchaserdd.collect()

[',apples,oranges', 'June,3,0', 'Robert,2,3', 'Lily,0,7', 'David,1,2']

In [18]:
purchasedf=spark.read.csv("/home/labuser/Desktop/Pandas_datasets/purchases.csv")
purchasedf.show()

+------+------+-------+
|   _c0|   _c1|    _c2|
+------+------+-------+
|  null|apples|oranges|
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



In [19]:
purchasedf_01 = spark.read.option("inferSchema",True).option("header",True).csv("/home/labuser/Desktop/Pandas_datasets/purchases.csv")
purchasedf_01.show()

+------+------+-------+
|   _c0|apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



23/09/22 08:15:31 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , apples, oranges
 Schema: _c0, apples, oranges
Expected: _c0 but found: 
CSV file: file:///home/labuser/Desktop/Pandas_datasets/purchases.csv


In [20]:
from pyspark.sql.types import StructType, StructField,IntegerType,StringType
moviesSchema=StructType([StructField("Rank",IntegerType(),True),
                     StructField("Title",StringType(),True),
                     StructField("Genre",StringType(),True)])

In [21]:
moviesDf = spark.read.schema(moviesSchema).option("header",True).csv("/home/labuser/Desktop/Pandas_datasets/IMDB-Movie-Data.csv")
moviesDf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)



In [22]:
moviesDf.show()

23/09/22 08:15:32 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 3
CSV file: file:///home/labuser/Desktop/Pandas_datasets/IMDB-Movie-Data.csv


+----+--------------------+--------------------+
|Rank|               Title|               Genre|
+----+--------------------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|
|   2|          Prometheus|Adventure,Mystery...|
|   3|               Split|     Horror,Thriller|
|   4|                Sing|Animation,Comedy,...|
|   5|       Suicide Squad|Action,Adventure,...|
|   6|      The Great Wall|Action,Adventure,...|
|   7|          La La Land|  Comedy,Drama,Music|
|   8|            Mindhorn|              Comedy|
|   9|  The Lost City of Z|Action,Adventure,...|
|  10|          Passengers|Adventure,Drama,R...|
|  11|Fantastic Beasts ...|Adventure,Family,...|
|  12|      Hidden Figures|Biography,Drama,H...|
|  13|           Rogue One|Action,Adventure,...|
|  14|               Moana|Animation,Adventu...|
|  15|            Colossal| Action,Comedy,Drama|
|  16|The Secret Life o...|Animation,Adventu...|
|  17|       Hacksaw Ridge|Biography,Drama,H...|
|  18|        Jason 

In [23]:
testDf = moviesDf.repartition(10)

testDf.rdd.getNumPartitions()

23/09/22 08:15:32 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 3
CSV file: file:///home/labuser/Desktop/Pandas_datasets/IMDB-Movie-Data.csv


10

In [24]:
testDf.write.csv("/home/labuser/Downloads/Output/out")

23/09/22 08:15:33 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 3
CSV file: file:///home/labuser/Desktop/Pandas_datasets/IMDB-Movie-Data.csv


AnalysisException: [PATH_ALREADY_EXISTS] Path file:/home/labuser/Downloads/Output/out already exists. Set mode as "overwrite" to overwrite the existing path.

In [25]:
spark_ui_url = f'{spark._jsc.sc().uiWebUrl().get()}/'
print("Spark UI URL:",spark_ui_url)

Spark UI URL: http://ip-172-31-10-64.ap-south-1.compute.internal:4040/


In [26]:
dsc=spark.sparkContext

In [27]:
dsc.uiWebUrl

'http://ip-172-31-10-64.ap-south-1.compute.internal:4040'

In [28]:
purchasedf_01.createOrReplaceTempView("purchase")

In [29]:
result=spark.sql("select * from purchase")

In [30]:
type(result)

pyspark.sql.dataframe.DataFrame

In [31]:
result.show()

+------+------+-------+
|   _c0|apples|oranges|
+------+------+-------+
|  June|     3|      0|
|Robert|     2|      3|
|  Lily|     0|      7|
| David|     1|      2|
+------+------+-------+



23/09/22 08:18:19 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , apples, oranges
 Schema: _c0, apples, oranges
Expected: _c0 but found: 
CSV file: file:///home/labuser/Desktop/Pandas_datasets/purchases.csv


In [32]:
from pyspark.sql.functions import *


In [33]:
testdf=moviesDf.sort(col("Title").desc())

In [34]:
testdf.show()

23/09/22 08:18:22 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 3
CSV file: file:///home/labuser/Desktop/Pandas_datasets/IMDB-Movie-Data.csv


+----+--------------------+--------------------+
|Rank|               Title|               Genre|
+----+--------------------+--------------------+
|  75|            Zootopia|Animation,Adventu...|
| 432|         Zoolander 2|              Comedy|
| 364|          Zombieland|Adventure,Comedy,...|
| 278|              Zodiac| Crime,Drama,History|
| 545|              Zipper|      Drama,Thriller|
| 407|    Zero Dark Thirty|Drama,History,Thr...|
| 793|Zack and Miri Mak...|      Comedy,Romance|
| 504|               Youth|  Comedy,Drama,Music|
| 986|       Your Highness|Adventure,Comedy,...|
| 838|You Don't Mess wi...|       Action,Comedy|
| 801|         Yoga Hosers|Comedy,Fantasy,Ho...|
| 206|      X: First Class|Action,Adventure,...|
| 627|X-Men: The Last S...|Action,Adventure,...|
| 163|X-Men: Days of Fu...|Action,Adventure,...|
|  33|   X-Men: Apocalypse|Action,Adventure,...|
| 269|X-Men Origins: Wo...|Action,Adventure,...|
| 969|             Wrecker|Action,Horror,Thr...|
| 564|      Wreck-It

In [35]:
df=moviesDf.select('Title','Rank')
df.show()

+--------------------+----+
|               Title|Rank|
+--------------------+----+
|Guardians of the ...|   1|
|          Prometheus|   2|
|               Split|   3|
|                Sing|   4|
|       Suicide Squad|   5|
|      The Great Wall|   6|
|          La La Land|   7|
|            Mindhorn|   8|
|  The Lost City of Z|   9|
|          Passengers|  10|
|Fantastic Beasts ...|  11|
|      Hidden Figures|  12|
|           Rogue One|  13|
|               Moana|  14|
|            Colossal|  15|
|The Secret Life o...|  16|
|       Hacksaw Ridge|  17|
|        Jason Bourne|  18|
|                Lion|  19|
|             Arrival|  20|
+--------------------+----+
only showing top 20 rows



In [36]:
testdf=moviesDf.withColumn('updated_list_ts',lit(datetime.datetime.now()))

NameError: name 'datetime' is not defined

In [37]:
testdf.show()

+----+--------------------+--------------------+
|Rank|               Title|               Genre|
+----+--------------------+--------------------+
|  75|            Zootopia|Animation,Adventu...|
| 432|         Zoolander 2|              Comedy|
| 364|          Zombieland|Adventure,Comedy,...|
| 278|              Zodiac| Crime,Drama,History|
| 545|              Zipper|      Drama,Thriller|
| 407|    Zero Dark Thirty|Drama,History,Thr...|
| 793|Zack and Miri Mak...|      Comedy,Romance|
| 504|               Youth|  Comedy,Drama,Music|
| 986|       Your Highness|Adventure,Comedy,...|
| 838|You Don't Mess wi...|       Action,Comedy|
| 801|         Yoga Hosers|Comedy,Fantasy,Ho...|
| 206|      X: First Class|Action,Adventure,...|
| 627|X-Men: The Last S...|Action,Adventure,...|
| 163|X-Men: Days of Fu...|Action,Adventure,...|
|  33|   X-Men: Apocalypse|Action,Adventure,...|
| 269|X-Men Origins: Wo...|Action,Adventure,...|
| 969|             Wrecker|Action,Horror,Thr...|
| 564|      Wreck-It

23/09/22 08:18:25 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 12, schema size: 3
CSV file: file:///home/labuser/Desktop/Pandas_datasets/IMDB-Movie-Data.csv


In [38]:
testdf.selectExpr('CAST(updated_list_ts AS DATE)AS test','Title').show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `updated_list_ts` cannot be resolved. Did you mean one of the following? [`Title`, `Genre`, `Rank`].; line 1 pos 5;
'Project [cast('updated_list_ts as date) AS test#176, Title#81]
+- Sort [Title#81 DESC NULLS LAST], true
   +- Relation [Rank#80,Title#81,Genre#82] csv


In [39]:
movieDf = spark.read.option("inferSchema",True).option("header",True).csv("/home/labuser/Desktop/Pandas_datasets/IMDB-Movie-Data.csv")
#movieDf.printSchema()
movieDf.dropDuplicates(["Director"]).count()

646

In [40]:
movieDf.count()

1000

In [41]:
ifDf = movieDf.withColumn("Rating_value",when(col("Rating")<=5,"Average").when((col("Rating")>5) & (col("Rating")<=8),"Good").when((col("Rating")>8) & (col("Rating")<=10),"Best").otherwise("Bad"))
ifDf.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|Rating_value|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|        Good|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|     7|485820|        

In [42]:
def concat_shell(column):
    return column+"_shell"

In [43]:
from pyspark.sql.functions import *
my_uds=udf(concat_shell,StringType())

In [44]:
test=movieDf.withColumn("new_col",my_uds(col("Title")))

In [45]:
test.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+--------------------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|             new_col|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+--------------------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|Guardians of the ...|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|        

In [49]:
cacDf = spark.read.option("inferSchema",True).option("header",True).csv("/home/labuser/Desktop/Pandas_datasets/IMDB-Movie-Data.csv")
cacDf.printSchema()

root
 |-- Rank: integer (nullable = true)
 |-- Title: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Director: string (nullable = true)
 |-- Actors: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Runtime (Minutes): string (nullable = true)
 |-- Rating: string (nullable = true)
 |-- Votes: string (nullable = true)
 |-- Revenue (Millions): double (nullable = true)
 |-- Metascore: double (nullable = true)



In [50]:
new = cacDf.cache()

In [51]:
new.show()

+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|Rank|               Title|               Genre|         Description|            Director|              Actors|                Year|Runtime (Minutes)|Rating| Votes|Revenue (Millions)|Metascore|
+----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----------------+------+------+------------------+---------+
|   1|Guardians of the ...|Action,Adventure,...|A group of interg...|          James Gunn|Chris Pratt, Vin ...|                2014|              121|   8.1|757074|            333.13|     76.0|
|   2|          Prometheus|Adventure,Mystery...|Following clues t...|        Ridley Scott|Noomi Rapace, Log...|                2012|              124|     7|485820|            126.46|     65.0|
|   3|               Split|   

In [52]:
customerDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/customer.csv")
lineitemDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/lineitem.csv")
nationDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/nation.csv")
ordersDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/orders.csv")
partDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/part.csv")
partsuppDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/partsupp.csv")
regionDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/region.csv")
supplierDf = spark.read.option("sep","\t").option("header",True).option("inferSchema",True).csv("/home/labuser/Downloads/Sales/supplier.csv")

In [55]:
joinDf= customerDf.join(ordersDf,customerDf.C_CUSTKEY==ordersDf.O_CUSTKEY,how='inner')
joinDf.show()

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|C_CUSTKEY|            C_NAME|           C_ADDRESS|C_NATIONKEY|        C_PHONE|C_ACCTBAL|C_MKTSEGMENT|           C_COMMENT|O_ORDERKEY|O_CUSTKEY|O_ORDERSTATUS|O_TOTALPRICE|O_ORDERDATE|O_ORDERPRIORITY|        O_CLERK|O_SHIPPRIORITY|           O_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+----------+---------+-------------+------------+-----------+---------------+---------------+--------------+--------------------+
|      370|Customer#000000370|          oyAPndV IN|         12|22-524-280-8721|  8982.79|   FURNITURE|ges. final packag...|         1|      370|            O|   172799.49| 1996-01-02|          5-LOW|Clerk#000000951|             0|nstructions sl

In [57]:
grpdg=ordersDf.groupBy("O_CLERK").sum("O_TOTALPRICE")
grpdg.show()

+---------------+------------------+
|        O_CLERK| sum(O_TOTALPRICE)|
+---------------+------------------+
|Clerk#000000024|        2078084.41|
|Clerk#000000288|        1431028.03|
|Clerk#000000236|        2571896.97|
|Clerk#000000093|3388791.1500000004|
|Clerk#000000124|        1683034.58|
|Clerk#000000011|2032237.1699999995|
|Clerk#000000114|1596777.3599999999|
|Clerk#000000022|         1250757.7|
|Clerk#000000474|1669808.3399999999|
|Clerk#000000723| 923212.7899999999|
|Clerk#000000903|2984974.0700000008|
|Clerk#000000398|        1799587.31|
|Clerk#000000744|         2078169.2|
|Clerk#000000912|1728371.5500000005|
|Clerk#000000644|         2357482.5|
|Clerk#000000766|        1941942.28|
|Clerk#000000674|         941649.11|
|Clerk#000000821|2854880.0399999996|
|Clerk#000000424|1509396.7800000003|
|Clerk#000000087|        1396516.85|
+---------------+------------------+
only showing top 20 rows

